In [1]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
import nltk
nltk.download('punkt')
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import geopandas as gpd
import urllib
import json
from dateutil.parser import parse
from datetime import datetime
import re
from collections import Counter

pd.options.display.max_columns = 120
pd.options.display.max_rows = 120
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/geoffperrin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
data = '/Users/geoffperrin/Desktop/bloomberg_work/bogota_project/data/'

In [3]:
xl = pd.ExcelFile(data + "SDQS/SDQS 2017-enejun.xlsx")

In [4]:
xl.sheet_names

[u'DB Recibidos']

In [5]:
df = xl.parse('DB Recibidos')
df = df[:1000]
df.head()

,Numero petición,Sector,Entidad,Entidad Nueva,Trasladado?,Entidad definitiva,Dependencia,Tema,Subtema,Localidad de los hechos,UPZ de los hechos,Barrio de los hecho,Estrato de los hechos,Punto atención,Canal,Tipo petición,Asunto,Fecha ingreso,estado,inicio terminos,cierre,dias gestión,Fecha registro,Nombre peticionario,Condición del peticionario,Localidad del ciudadano,UPZ del ciudadano,Barrio del ciudadano,Estrato del ciudadano,Tipo de Peticionario,Opciones de Petición,Entidad que recibe,Entidad que traslada
0,2112017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,SERVICIOS PUBLICOS,ATENCION Y SERVICIO A LA CIUDADANIA,4 - SAN CRISTOBAL,32 - SAN BLAS,BELLAVISTA SUR,2.0,NaN,WEB,DERECHO DE PETICIÓN DE INTERÉS PARTICULAR,BUENAS TARDES ES PARA SOLICITAR LA REPARACION ...,2017-01-02,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03,2017-01-23 12:59:50.415,14.0,2017-01-03,JERSON ANDRES ROJAS RODRIGUEZ,NaN,4 - SAN CRISTOBAL,32 - SAN BLAS,BELLAVISTA SUR,2.0,Natural,En nombre propio,NaN,NaN
1,2992017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,SERVICIOS PUBLICOS,VISITAS TECNICAS,12 - BARRIOS UNIDOS,21 - LOS ANDES,ENTRERIOS,4.0,NaN,WEB,RECLAMO,INSTALACION DE MEDIDOR EL 8 DE NOVIEMBRE DE 20...,2017-01-02,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03,2017-01-24 14:26:01.929,15.0,2017-01-03,AMANDA OLIVA BOTERO SANCHEZ,ADULTO MAYOR,12 - BARRIOS UNIDOS,21 - LOS ANDES,ENTRERIOS,4.0,Natural,En nombre propio,NaN,NaN
2,3622017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,SERVICIOS PUBLICOS,DISPONIBILIDAD DE SERVICIO,2 - CHAPINERO,99 - CHAPINERO,CHAPINERO NORTE,5.0,NaN,WEB,DERECHO DE PETICIÓN DE INTERÉS PARTICULAR,EMPRESA DE ACUEDUCTO Y ALCANTARILLADO DE BOGOT...,2017-01-02,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03,2017-02-02 11:20:55.660,22.0,2017-01-03,JAIME MONCAYO ARENAS,ADULTO MAYOR,2 - CHAPINERO,99 - CHAPINERO,CHAPINERO NORTE,5.0,Natural,En nombre propio,NaN,NaN
3,3772017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,AMBIENTE,"MANTENIMIENTO DE SUMIDEROS, REJILLAS DE ALCANT...",2 - CHAPINERO,97 - CHICO LAGO,ANTIGUO COUNTRY,5.0,NaN,WEB,RECLAMO,FAVOR REPONER LA REJILLA DE SUMNIDERO QUE SE E...,2017-01-02,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03,2017-02-02 11:55:16.434,22.0,2017-01-03,SERGIO VALDES BELTRAN,NaN,NaN,NaN,NaN,5.0,Natural,En nombre propio,NaN,NaN
4,3862017,HABITAT,ACUEDUCTO - EAB,NaN,NaN,ACUEDUCTO - EAB,DIRECCION DE SERVICIO ADMINISTRATIVOS,SERVICIOS PUBLICOS,RECUPERACIÓN DE ESPACIO PUBLICO POR OBRAS DE L...,1 - USAQUEN,14 - USAQUEN,USAQUEN,6.0,NaN,WEB,QUEJA,ME REFIERO A SU COMUNICACION S-2016-245954 LA ...,2017-01-02,SOLUCIONADO - POR RESPUESTA DEFINITIVA,2017-01-03,2017-01-16 10:47:02.935,9.0,2017-01-03,SERGIO VALDES BELTRAN,NaN,NaN,NaN,NaN,5.0,Natural,En nombre propio,NaN,NaN


In [6]:
df['Asunto'].iloc[0]

u'BUENAS TARDES ES PARA SOLICITAR LA REPARACION  DEL REGISTRO DE CORTE DE MI CONTADOR DE AGUA, MUCHAS GRACIAS\n'

In [7]:
es_blob = TextBlob(df['Asunto'].iloc[0])

In [8]:
es_blob.translate(to='en')

TextBlob("GOOD AFTERNOON IS TO REQUEST THE REPAIR OF THE CUTTING REGISTER OF MY WATER COUNTER, THANK YOU")

translation should be - 'good afternoon, (the complaint) is to request the repair of the court record of my water meter, thank you'

In [9]:
en_blob = es_blob.translate(to='en')

In [10]:
en_blob.sentiment.polarity

0.04999999999999999

### sentiment / naive bayes help gathered from here:
https://www.dataquest.io/blog/naive-bayes-tutorial/

In [11]:
def count_text(text):
    # Split text into words based on whitespace.  Simple but effective.
    words = re.split("\s+", text)
    # Count up the occurence of each word.
    return Counter(words)

In [12]:
def length_text(text):
    # Split text into words based on whitespace.  Simple but effective.
    words = re.split("\s+", text)
    # Count up the occurence of each word.
    return len(words)

In [13]:
quejas_length = []
quejas_sentiment_polarity = []
#text_ is how each point should be labeled - ideally I'd want to use category or something...
text_ = []
for i in df['Asunto']:
    es_blob = TextBlob(i)
    en_blob = es_blob.translate(to='en')
    print en_blob.sentiment.polarity
    print length_text(i)
    quejas_sentiment_polarity.append(en_blob.sentiment.polarity)
    quejas_length.append(length_text(i))

0.05
19
0.118445839874
263
0.0312254033845
638
-0.2
50
0.0666666666667
43
-0.0546743697479
424
0.362083333333
98
0.295238095238
102
-0.09375
36
0.0
22
0.261111111111
36
0.0
33
-0.222222222222
42
0.0
5
0.05
63
0.0555555555556
84
0.401666666667
69
0.0
41
-0.05
35
0.191666666667
25
0.14375
119
-0.2
17
-0.3
8
0.0
50
0.0
15
0.100595238095
386
0.0
28
0.111111111111
123
0.222222222222
171
0.2
40
-0.0303636363636
192
-0.133333333333
25
0.0426315789474
275
0.233333333333
90
0.3
58
-0.0121212121212
103
0.255555555556
70
0.0988461538462
144
0.122222222222
45
-0.15
42
0.025
113
0.09375
95
0.0
19
-0.1
38
-0.6
8
-0.6
12
-0.6
8
0.26
19
0.285714285714
18
0.0190476190476
100
0.123917748918
474
-0.0833333333333
118
0.0
7
0.0
46
0.00833333333333
62
0.208333333333
33
-0.0576041666667
195
-0.0777777777778
90
0.11
55
0.28875
102
0.144444444444
38
0.211712361712
95
0.0
9
-0.0125
78
0.11829004329
345
0.0
5
0.35
155
-0.010101010101
15
0.129672232529
190
-0.188888888889
46
-0.25
8
-0.145833333333
11
-0.16666666

### Plotting Sentiment Score against complaint (queja) length

In [24]:
trace1 = go.Scatter(x=quejas_length, y=quejas_sentiment_polarity, marker={'color': 'red', 'size': "6"}, 
                    mode="markers", name='1st Trace')
                                               
data=go.Data([trace1])
layout=go.Layout(title="Veeduría Quejas Plot", xaxis={'title':'Number of Words in Queja'}, yaxis={'title':'Sentiment Score'})
figure=go.Figure(data=data,layout=layout)
py.iplot(figure, filename='pyguide_1')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~gperrin12/0 or inside your plot.ly account where it is named 'pyguide_1'
